In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [4]:
imgPaths = glob('image/*.jpg')

In [5]:
allinvoices = pd.DataFrame(columns=['id','text'])

for imgPath in  tqdm(imgPaths,desc='invoice'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)
    # extract data and text 
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(int)

    useFulData = df.query('conf >= 30')

    # Dataframe
    invoice = pd.DataFrame()
    invoice['text'] = useFulData['text']
    invoice['id'] = filename
    
    # concatenation
    allinvoices = pd.concat((allinvoices,invoice))

invoice: 100%|██████████████████████████████████| 60/60 [40:02<00:00, 40.03s/it]


In [6]:
allinvoices.to_csv('all_invoice.csv',index=False)